# Real Optical Flow Torus Data Analysis

In [1]:
#Import functions for sampling and computing characteristic classes
from dreimac import CircularCoords
import circle_bundles as cb
import optical_flow as of
import synthetic as sy

cb.attach_bundle_viz_methods()



#For saving
folder_path = '/Users/bradturow/Desktop/Diagrams/Paper Diagrams/'

# Get A Sample of Optical Flow Patches From The Sintel Dataset

## Load Preprocessed Sample

In [2]:
import pickle 
import pandas as pd

#Load the data
with open('/Users/bradturow/Desktop/Circle Bundle Code/HC20_Flow_Patches.pkl', 'rb') as f:
    patch_df =pd.read_pickle(f)

#    density_options = [10, 50, 100, 200, 300, 1000]    

#Get the data in X(k,p)
k = 1500
p = 0.50
column = 'density_' + str(k)
patch_df = patch_df.sort_values(by = column, ascending = False)
N = int(p*len(patch_df))
data = np.vstack(patch_df['patch'])[:N]

print(f'Sample contains {len(data)} high-contrast patches.')

#View a sample of the data
patch_vis = make_patch_visualizer()
fig = show_data_vis(data, patch_vis, sampling_method = None, max_samples = 30)
plt.show()

Sample contains 125000 high-contrast patches.


NameError: name 'make_patch_visualizer' is not defined

## ...Or Collect A New Sample And Preprocess

In [ ]:
from optical_flow.flow_processing import *

patches_per_frame = 400
folder_path = "/Users/bradturow/Desktop/TDA/MPI-Sintel-complete/training/flow"

patch_df, file_paths = get_patch_sample(
    folder_path,
    patches_per_frame = patches_per_frame,
    d = 3)

print('')
print(f'{len(patch_df)} optical flow patches sampled')

#Downsample if necessary
max_samples = 400000
if len(patch_df) > max_samples:
    patch_df = patch_df.sample(n=max_samples)


In [ ]:
#Preprocess the sample
hc_frac = 0.2
max_samples = 50000
k = [300]

patch_df = preprocess_flow_patches(
    patch_df,
    hc_frac = hc_frac,
    max_samples = max_samples,
    k_list = k)

print('preprocessing complete.')

In [ ]:
#Save the data
folder_path = '/Users/bradturow/Desktop/Diagrams/Paper Diagrams/'
file_name = '3x3_pp_hc_opt_flow.pkl'
save_path = folder_path + file_name
with open(save_path, 'wb') as f:
    pickle.dump(patch_df, f)

    

In [ ]:
#Keep only the densest patches
p = 0.5
n_samples = int(p*len(patch_df))
data = np.vstack(patch_df['patch'])[:n_samples] #Data is already sorted in decreasing order by density
print(f'Downsampled to {len(data)} patches')

In [ ]:
#Show a PCA visualization of the dataset
show_pca(data)

In [ ]:
#Run Ripser on the dataset
from ripser import ripser
from persim import plot_diagrams

diagrams = ripser(data, maxdim = 2, n_perm = 500)['dgms']
plot_diagrams(diagrams, show=True)    

# Bundle Analysis

In [ ]:
from optical_flow.contrast import *
#Compute the base projection and directional strength for each patch
predom_dirs, ratios = get_predominant_dirs(data)

#Map RP1 angles to points on the unit circle (for visualization)
base_points = np.array([np.cos(2*predom_dirs), np.sin(2*predom_dirs)]).T

#Construct a cover of the base space
n_landmarks = 16
lmk_angs = np.linspace(0, 2*np.pi,n_landmarks, endpoint= False)
landmarks = np.array([np.cos(lmk_angs), np.sin(lmk_angs)]).T
overlap = 1.99

radius = overlap* np.pi/n_landmarks

cover = MetricBallCover(base_points, landmarks, radius, metric = S1_dist2)
cover_data = cover.build()

#Show a summary of the construction
summ = cover.summarize(plot = True)

In [ ]:
#View a sample of the dataset arranged by predominant flow direction
n_samples = 8

label_func = [fr"$\theta = {np.round(pred/np.pi, 2)}$" + r"$\pi$" for pred in predom_dirs]
file_name = 'opt flow sample.pdf'
save_path = folder_path + file_name
save_path = None
fig = show_data_vis(data, patch_vis, label_func = label_func, angles = predom_dirs, sampling_method = 'angle', max_samples = n_samples, save_path = save_path)
plt.show()


In [ ]:
#Try running circular coordinates on a single open set
j= 0
n_local_landmarks = 200
prime = 43
fiber_data = data[cover.U[j]]
patch_angles, n_warnings, n_lmks = compute_circular_coords_dreimac(fiber_data, n_landmarks_init = n_local_landmarks, CircularCoords_cls = CircularCoords)

print(n_warnings, n_lmks)
show_pca(fiber_data, colors = patch_angles)




In [ ]:
#Construct local circular coordinates and model transitions as O(2) matrices
bundle = build_bundle(
    data,
    cover,
    CircularCoords_cls=CircularCoords,
    show=True,
)


In [ ]:
#View the correlations between local circular coordinates on overlaps
file_name = 'real_oft_correlations2.pdf'
save_path = folder_path + file_name
save_path = None
fig = bundle.compare_trivs(ncols = 4, save_path = save_path)
plt.show()

In [ ]:
#Get a global coordinatization
triv_result = bundle.get_global_trivialization()
print('Global coordinates computed.')

In [ ]:
#Show an interactive visualization of the bundle colored by fiber coordinate
fig = bundle.show_bundle(get_dist_mat = get_dist_mat, colors = triv_result.F)
plt.show()

In [ ]:
#Show a visualization of the nerve labeled with SW1

#Compute a potential for the restricted orientation class
subcomplex = bundle.get_max_trivial_subcomplex()
edges = subcomplex.kept_edges
Omega = bundle.classes.cocycle_used.restrict(edges)
phi_vec = Omega.orient_if_possible(edges)[2]
phi = {lmk: phi_vec[lmk] for lmk in range(n_landmarks)}
omega = bundle.classes.omega_O1_used

save_path = folder_path + 'real_oft_nerve.pdf'
save_path = None
fig = bundle.show_circle_nerve(omega = omega, phi = phi, save_path = save_path)
plt.show()



In [ ]:
#Show an interactive visualization of the bundle, with data colored by global fiber coordinate
fig = bundle.show_bundle(get_dist_mat = get_dist_mat, colors=triv_result.F)
plt.show()

## Restrict To High-Directionality Data 

In [ ]:
#Set up a bundle with just the high-directionality data
thresh = 0.8
high_inds = ratios > thresh
print(f'{np.sum(high_inds)} high-directionality patches')

high_cover = MetricBallCover(base_points[high_inds], landmarks, radius, metric = S1_dist2)
high_cover_data = high_cover.build()

#Show a summary of the construction
high_summ = high_cover.summarize(plot = True)

In [ ]:
#Construct local circular coordinates and model transitions as O(2) matrices
high_bundle = build_bundle(
    data[high_inds],
    high_cover,
    CircularCoords_cls=CircularCoords,
    show=True,
)


In [ ]:
file_name = 'Real Opt Flow Triv Correlations.pdf'
save_path = folder_path + file_name
save_path = None
fig = high_bundle.compare_trivs(ncols = 4, save_path = save_path)
plt.show()

In [ ]:
high_triv_result = high_bundle.get_global_trivialization()
print('global coordinates computed.')

In [ ]:
#Show a recovered patch diagram
per_row = 5
per_col = 9
coords = np.array([predom_dirs[high_inds], high_triv_result.F]).T
file_name = 'Real_OFT_Diagram_High.pdf'
save_path = folder_path + file_name
save_path = None

fig = lattice_vis(
    high_bundle.data,
    coords,
    patch_vis,
    per_row=per_row,
    per_col = per_col,
    figsize=12,
    thumb_px=350,   
    dpi=350, 
    save_path = save_path,
    padding = 0
)

plt.show()


In [ ]:
#Get a visualization of the low-directionality data
thresh = 0.7
low_inds = ratios < thresh
print(f'{np.sum(low_inds)} low-directionality patches')
low_data = data[low_inds]

#Show a recovered patch diagram
per_row = 5
per_col = 9
coords = np.array([predom_dirs[low_inds], triv_result.F[low_inds]]).T
file_name = 'Real_OFT_Diagram_Low.pdf'
save_path = folder_path + file_name
save_path = None

fig = lattice_vis(
    low_data,
    coords,
    patch_vis,
    per_row=per_row,
    per_col = per_col,
    figsize=12,
    thumb_px=350,   
    dpi=350,
    save_path = save_path,
    padding = 0
)

plt.show()

